In [133]:
# import packages
from tkinter import *
from tkinter import ttk
from tkinter import scrolledtext
import subprocess
import serial
import serial.tools.list_ports
import time
import threading

# important variable for stopping
halt = False

# defines a function that gets devices plugged into serial ports
# if one's name is 'port authority', we set that port 
def port_authority():
    ports = list(serial.tools.list_ports.comports())
    # this list is 2D
    for p in ports:
        # basically, if it says 'prolific' in the right side
        # give us the name of the port (on the left side)
       if "Prolific" in p[1]:
          return p[0]

In [134]:
# ~~ TODO ~~
# create a widget that dislplays output terminal in the center (use tkterminal!)
# create a widget that displays current time (use threads!)
# make it actually loop????

# ~~ DONE! ~~
# make a enlution time change entry
# use threads to make previous method run, pause, loop and escape at correct times! 

In [135]:
# runs our function and defines a variable as the port named 'prolific'
port=port_authority()
# baud rate is 9600, timeout is 1 second
ser = serial.Serial(port,9600,timeout=0.5)

In [136]:
# defines all useful functions

def Wait(i):
    t = int(i)
    time.sleep(t)


def Start():
    ser.write(b'*GOA\n')
    start_time = time.perf_counter()
    print("Starting Run")
    return start_time


'''
test function that causes
something to happen when you
press start

def RunSample():
    ser.write(b'TO\n')
'''

def Rinse30(st):
    ser.write(b'2GOB\n')
    print("Rinsing...")
    Wait(30)
    current_time = time.perf_counter()
    output_time = current_time - st
    print("Rinsed! Current time: " + str(output_time))



def Rinse45(st):
    ser.write(b'1GOA\n')
    print("Rinsing...")
    Wait(45)
    current_time = time.perf_counter()
    output_time = current_time - st
    print("Rinsed! Current time: " + str(output_time))



def Load(st,i):
    t = int(i)
    ser.write(b'*GOB\n')
    print("Loading...")
    Wait(t)
    current_time = time.perf_counter()
    output_time = current_time - st
    print("Loaded! Current time: " + str(output_time))


def Elute(st,i):
    t = int(i)
    ser.write(b'*GOA\n')
    print("Eluting...")
    Wait(t)
    current_time = time.perf_counter()
    output_time = current_time - st
    print("Eluted! Time elapsed: " + str(output_time))
    #print("Ready for next run.")
    
def Stop():
    global halt
    halt = True
    #StartButton["state"] = NORMAL
    StopButton["state"] = DISABLED
    print("Stopping at next avaliable moment")

In [137]:
#big function...
def RunSample(load, elute):
    # remove when done
    #print("Make sure both valves are at A.")
    #ser.write(b'TO\n')
    
    i = 1
    Wait(i)
    global halt
    global loopvar
    start_time = 0
        
    # makes sure all valves are at position A
    start_time = Start()
       
    if halt == False:
        # rinse for 30 seconds
        Rinse30(start_time)
    else:
        print("Stopping")
        if loopvar == True:
            loopvar = False
        halt = False
        return
        
    if halt == False:
        # load for t seconds (t is user defined time)
        Load(start_time, load)
    else:
        print("Stopping")
        if loopvar == True:
            loopvar = False
        halt = False
        return
    
    if halt == False:
        # rinse for 45 seconds
        Rinse45(start_time)
    else:
        print("Stopping")
        if loopvar == True:
            loopvar = False
        halt = False
        return

    # elute for an amount of time
    Elute(start_time, elute)

    
#def test():
#    Wait(5)
#    print("MEOW")
    
def one(l, e):
    global loopvar
    loop = loopvar.get
    thread2 = threading.Thread(target=RunSample, args=(l, e))
    thread2.start()
    thread2.join()
    loop = loopvar.get
    
    while loop == True:
        thread2.start(l, e)
        thread2.join()
        loop = loopvar.get
    else:  
        print("Ready for next run")
        StartButton["state"] = NORMAL
        StopButton["state"] = DISABLED
        return
    
def CurrentCount():
    # write a function that can be used to display current time
    return

    # uses threads
def StartRoutine(l, e):
    L = int(l.get())
    E = int(e.get())
    StartButton["state"] = DISABLED
    StopButton["state"] = NORMAL
    thread1 = threading.Thread(target=one, args=(L, E))
    
    thread1.start()
    
    
    #print("Ready for next run")
    
    
    

In [138]:


count_thread = threading.Thread(target=CurrentCount)
#routine_thread = threading.Thread(target=RunSample, args(Load, Elute, Loop, CarryOn))


# sets the main application window and gives a title
root = Tk()
root.title("Valve Controller")

# creates a mainframe widget, which holds contents of UI
mainframe = ttk.Frame(root, padding="0.2i")
# this is for the middle widget
frame = ttk.Frame(mainframe, borderwidth=5, relief="ridge", width=300, height=150)

# needed for loop button
global loopvar
loopvar = BooleanVar(value=False)

# creating basic text labels
Welcome = ttk.Label(mainframe, text="Welcome!")
LoadTime = ttk.Label(mainframe, text="Load Time (s):")
EluteTime = ttk.Label(mainframe, text="Elute Time (s):")
#Ready = ttk.Label(mainframe, text="Ready?")
CurrentTime = ttk.Label(mainframe, text="Current Time (s):")


# creating an entry widget AND default values for text entry
loadtime = StringVar(value="120") 
loadtime_entry = ttk.Entry(mainframe, width=7, textvariable=loadtime)
elutetime = StringVar(value="240")
elutetime_entry = ttk.Entry(mainframe, width=7, textvariable=elutetime)

# creating the button widgets
StartButton = ttk.Button(mainframe, text="Start", command= lambda: StartRoutine(loadtime, elutetime))
StopButton = ttk.Button(mainframe, text="Stop", command=Stop)
LoopButton = ttk.Checkbutton(mainframe, text="Loop?", variable=loopvar, onvalue=True)
StopButton["state"] = DISABLED

# output widgets
currenttime = StringVar()
CurrentTime_output = ttk.Label(mainframe, textvariable=currenttime)

# terminal output in center



# places a grid inside our application window and places things in the grid
mainframe.grid(column=0, row=0, sticky=(N, S, E, W))

frame.grid(column=1,row=1,sticky=(N, W, E, S))
Welcome.grid(column=0,row=0,sticky=(E,S))
LoadTime.grid(column=0,row=2,sticky=E)
EluteTime.grid(column=0,row=3,sticky=E)
#Ready.grid(column=2,row=0,sticky=(S))
CurrentTime.grid(column=2,row=0,sticky=S)
loadtime_entry.grid(column=1, row=2, sticky=W)
elutetime_entry.grid(column=1, row=3, sticky=W)
StartButton.grid(column=2,row=2, sticky=S)
StopButton.grid(column=2,row=3, sticky=N)
LoopButton.grid(column=0, row=1, sticky=E)
CurrentTime_output.grid(column=2,row=1,sticky=N)



# tells it our grid should expand when the application is resized
root.columnconfigure(0, weight=1)
root.rowconfigure(0, weight=1)
mainframe.columnconfigure(0, weight=1)
mainframe.columnconfigure(1, weight=1)
mainframe.columnconfigure(2, weight=1)
mainframe.rowconfigure(0, weight=1)
mainframe.rowconfigure(1, weight=1)
mainframe.rowconfigure(2, weight=1)

# adding padding to all widgets
for child in mainframe.winfo_children():
    child.grid_configure(padx=5, pady=5)

# start the program basically
root.mainloop()


Starting Run
Rinsing...
Rinsed! Current time: 30.013209499999903
Loading...
Loaded! Current time: 31.01769289999993
Rinsing...
Rinsed! Current time: 76.0261829000001
Eluting...
Eluted! Time elapsed: 77.02778280000075
Ready for next run


In [139]:
# we have to close the connection so it doesn't throw an error when we rerun the script
ser.close()